In [1]:
import pandas as pd
import numpy as numpy

# Armado de sets de entrenamiento

In [2]:
train = pd.DataFrame()

In [3]:
for ventana in range(1,4):
    
    features_pageviews = pd.read_pickle("../data/features/ventana_{}_pageviews.pkl".format(ventana))
    features_conversion = pd.read_pickle("../data/features/ventana_{}_conversion.pkl".format(ventana))
    
    label = pd.read_pickle("../data/labels/labels_trimestre_{}.pkl".format(ventana+1))

    train_actual = label.merge(features_pageviews,left_index = True, right_index = True, how = 'left')
    train_actual = train_actual.merge(features_conversion,left_index = True, right_index = True, how = 'left')
    
    train = train.append(train_actual)

### Formateo los datos

#### Columnas categoricas

In [16]:
columnas_categoricas = [  'page_mas_vista_en_mes_1',
       'page_mas_vista_en_mes_2', 'page_mas_vista_en_mes_3']

In [21]:
train[columnas_categoricas] = train[columnas_categoricas].fillna(-1)
train[columnas_categoricas] = train[columnas_categoricas].astype("int64")
train[columnas_categoricas] = train[columnas_categoricas].astype("category")

#### Columnas con valores numericos

In [15]:
columnas_contar = ['cantidad_paginas_vistas_trimestre',
       'cant_paginas_vistas_en_mes_1', 'cant_paginas_vistas_en_mes_2',
       'cant_paginas_vistas_en_mes_3']

In [23]:
train[columnas_contar] = train[columnas_contar].fillna(0)
train[columnas_contar] = train[columnas_contar].astype("int64")

#### Columnas booleanas

In [17]:
columnas_booleanas = ['convirtio_en_mes_1', 'convirtio_en_mes_2', 'convirtio_en_mes_3',
       'convirtio_multiple_en_mes_1', 'convirtio_multiple_en_mes_2',
       'convirtio_multiple_en_mes_3']

In [18]:
train["convirtio"] = train["convirtio"] * 1

In [24]:
train[columnas_booleanas] = train[columnas_booleanas].fillna(0)
train[columnas_booleanas] = train[columnas_booleanas].astype("int8")

---

In [28]:
train.head()

,convirtio,cantidad_paginas_vistas_trimestre,cant_paginas_vistas_en_mes_1,cant_paginas_vistas_en_mes_2,cant_paginas_vistas_en_mes_3,page_mas_vista_en_mes_1,page_mas_vista_en_mes_2,page_mas_vista_en_mes_3,convirtio_en_mes_1,convirtio_en_mes_2,convirtio_en_mes_3,convirtio_multiple_en_mes_1,convirtio_multiple_en_mes_2,convirtio_multiple_en_mes_3
USER_ID,,,,,,,,,,,,,,
6574,0,1141,524,364,253,10,10,10,0,0,0,0,0,0
2639,1,671,261,179,231,3,5,3,0,0,0,0,0,0
3898,0,219,88,50,81,85,85,27,0,0,0,0,0,0
7996,0,383,95,178,110,2,2,2,0,0,0,0,0,0
6111,0,425,208,93,124,3,39,3,0,0,0,0,0,0


In [29]:
train.to_pickle("../data/train_completo.pkl")